In [3]:
import streamlit as st
from dotenv import load_dotenv
load_dotenv()

True

In [12]:
from langchain_community.document_loaders import PyPDFLoader,PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

In [3]:
pdf_path = "/mnt/d/LANGCHAIN_UDEMY/usecase/PDF-Chat App/MobileNets Efficient Convolutional Neural Networks for Mobile Vision.pdf"

In [8]:
pip install pypdf

  Using cached pypdf-4.3.1-py3-none-any.whl.metadata (7.4 kB)
Using cached pypdf-4.3.1-py3-none-any.whl (295 kB)
Note: you may need to restart the kernel to use updated packages.


In [18]:
pdf_path = "Data"
if os.path.exists(pdf_path) and len(os.listdir(pdf_path))>0: 
    loader = PyPDFDirectoryLoader(pdf_path)
else:
    print("Pass")    

Pass


In [20]:
from langchain import hub
hub.pull("langchain-ai/retrieval-qa-chat")

ChatPromptTemplate(input_variables=['context', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'langchain-ai', 'lc_hub_repo': 'retrieval-qa-chat', 'lc_hub_commit_hash': 'b60afb6297176b022244feb83066e10ecadcda7b90423654c4a9d45e7a73cebc'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='Answer any use questions based solely on the context below:\n\n<context>\n{context}\n</context>')), MessagesPlaceholder(variable_name='chat_history', optional=True), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])

In [8]:
loader = PyPDFLoader(pdf_path)
raw_documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 600,chunk_overlap = 50)
documents = text_splitter.split_documents(raw_documents)

In [18]:
# Create instance of OpenAI LLM
llm = ChatOpenAI(temperature=0.1, verbose=True)
embeddings = OpenAIEmbeddings()

In [44]:
class PdfChatApp():
    def __init__(self,file_path: str, embeddings, llm, vectorstore_dir) -> None:
        self.file_path = file_path
        self.embeddings = embeddings
        self.vectorstores_dir = vectorstore_dir
        self.search_type = "similarity"
        self.top_n = 3
        self.llm = llm
        pass
    def data_ingestion(self,file_path):
        # Create and load PDF Loader
        loader = PyPDFLoader(file_path)
        raw_documents = loader.load()
        print(f"Loaded {len(raw_documents)} documents")
        # Split Text from pdf 
        text_splitter = RecursiveCharacterTextSplitter(chunk_size = 600,chunk_overlap = 50)
        documents = text_splitter.split_documents(raw_documents)
        return documents
    def get_vector_store(self, docs, embeddings, vectorstore_dir):
        vectorstore = FAISS.from_documents(docs, embeddings)
        vectorstore.save_local(vectorstore_dir)
        return None
    
    def load_retriever(self, embeddings,vectorstore_dir):
        retriever = (
            FAISS
            .load_local(vectorstore_dir,embeddings,allow_dangerous_deserialization=True)
            .as_retriever(search_type = self.search_type, search_kwargs = {"k":self.top_n})
        )
        return retriever
    def get_conversational_chain(self):
        docs = self.data_ingestion(file_path=self.file_path)
        _ = self.get_vector_store(
            docs= docs,
            embeddings = self.embeddings,
            vectorstore_dir = self.vectorstores_dir
        )
        retriever = self.load_retriever(self.embeddings,self.vectorstores_dir)
        # 2. Incorporate the retriever into a question-answering chain.
        system_prompt = (
            "You are an assistant for question-answering tasks. "
            "Answer the question as detailed as possible from the provided context, make sure to provide all the details" 
            "if the answer is not in provided context just say, answer is not available in the context, don't provide the wrong answer"
            "\n\n"
            "{context}"
        )

        retrieval_qa_template = ChatPromptTemplate.from_messages(
            [
                ("system", system_prompt),
                ("human", "{input}"),
            ]
        )

        # retrieval_qa_template = PromptTemplate(template = template, input_variables = ["context", "question"])
        document_chain = create_stuff_documents_chain(self.llm, retrieval_qa_template)
        retrieval_chain = create_retrieval_chain(
            retriever=retriever, combine_docs_chain=document_chain
        )
        
        return retrieval_chain
    def get_llm_response(self,chain, question):
        return chain.invoke({"input": question})['answer']
    
    def get_llm_response(self,chain, question):
        return chain.invoke({"input": question})['answer']
        

In [ ]:
import os
import shutil
import os ,shutil
def clear_folder(folder_path):
    if os.path.exists(folder_path):
        # Remove all the files in the directory
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            try:
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.unlink(file_path)
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
            except Exception as e:
                print(f"Failed to delete {file_path}. Reason: {e}")
class PdfChatApp():
    def __init__(self, embeddings, llm, vectorstore_dir) -> None:
        self.file_path = None
        self.embeddings = embeddings
        self.vectorstores_dir = vectorstore_dir
        self.search_type = "similarity"
        self.top_n = 3
        self.llm = llm
        pass
    def store_files(self,files):
        current_directory = os.path.dirname(os.path.abspath(__file__))
        UPLOAD_FOLDER = os.path.join(current_directory, "RAW_DATA")
        clear_folder(UPLOAD_FOLDER)
        if not os.path.exists(UPLOAD_FOLDER):
            os.makedirs(UPLOAD_FOLDER,exist_ok=True)  
        for file in files:
            file = file.name
            file_name = file.split(os.sep)[-1]
            shutil.copy(file,os.path.join(UPLOAD_FOLDER,file_name))
        self.file_path =  UPLOAD_FOLDER
        return None   
    def data_ingestion(self,file_path):
        # Create and load PDF Loader
        loader = PyPDFLoader(file_path)
        raw_documents = loader.load()
        print(f"Loaded {len(raw_documents)} documents")
        # Split Text from pdf 
        text_splitter = RecursiveCharacterTextSplitter(chunk_size = 600,chunk_overlap = 50)
        documents = text_splitter.split_documents(raw_documents)
        return documents
    def get_vector_store(self, docs, embeddings, vectorstore_dir):
        vectorstore = FAISS.from_documents(docs, embeddings)
        vectorstore.save_local(vectorstore_dir)
        return None
    
    def load_retriever(self, embeddings,vectorstore_dir):
        retriever = (
            FAISS
            .load_local(vectorstore_dir,embeddings,allow_dangerous_deserialization=True)
            .as_retriever(search_type = self.search_type, search_kwargs = {"k":self.top_n})
        )
        return retriever
    def get_conversational_chain(self):
        docs = self.data_ingestion(file_path=self.file_path)
        _ = self.get_vector_store(
            docs= docs,
            embeddings = self.embeddings,
            vectorstore_dir = self.vectorstores_dir
        )
        retriever = self.load_retriever(self.embeddings,self.vectorstores_dir)
        # 2. Incorporate the retriever into a question-answering chain.
        system_prompt = (
            "You are an assistant for question-answering tasks. "
            "Answer the question as detailed as possible from the provided context, make sure to provide all the details" 
            "if the answer is not in provided context just say, answer is not available in the context, don't provide the wrong answer"
            "\n\n"
            "{context}"
        )

        retrieval_qa_template = ChatPromptTemplate.from_messages(
            [
                ("system", system_prompt),
                ("human", "{input}"),
            ]
        )

        # retrieval_qa_template = PromptTemplate(template = template, input_variables = ["context", "question"])
        document_chain = create_stuff_documents_chain(self.llm, retrieval_qa_template)
        retrieval_chain = create_retrieval_chain(
            retriever=retriever, combine_docs_chain=document_chain
        )
        
        return retrieval_chain
    def get_llm_response(self,chain, question):
        return chain.invoke({"input": question})['answer']
    
    def get_llm_response(self,chain, question):
        return chain.invoke({"input": question})['answer']
        

In [2]:
import os 
import shutil
def clear_folder(folder_path):
    if os.path.exists(folder_path):
        # Remove all the files in the directory
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            try:
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.unlink(file_path)
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
            except Exception as e:
                print(f"Failed to delete {file_path}. Reason: {e}")
vectorstore_dir = "vectordb"
if not os.path.exists(vectorstore_dir):
    os.makedirs(vectorstore_dir,exist_ok=True)
clear_folder(vectorstore_dir)

In [45]:
pdfchatapp = PdfChatApp(pdf_path,embeddings,llm,"PDF-Chat App/vectordb")
generator_chain = pdfchatapp.get_conversational_chain()
query = "What is MobileNet architrecture ?"


Loaded 2 documents


In [46]:
pdfchatapp.get_llm_response(generator_chain,query)

'The MobileNet architecture is based on depthwise separable convolutions. It consists of core layers built on depthwise separable filters, where the standard convolutional filters are replaced by two layers: depthwise convolution and pointwise convolution. The network structure of MobileNet primarily utilizes depthwise separable convolutions, except for the first layer which is a full convolution. Each layer in MobileNet is followed by batch normalization and ReLU nonlinearity, except for the final fully connected layer which feeds into a softmax layer for classification. The architecture allows for easy exploration of network topologies to find an efficient network design. Additionally, MobileNet architecture includes model shrinking hyperparameters like width multiplier and resolution multiplier for optimization.'

In [47]:
import tempfile

In [49]:
tempfile.TemporaryDirectory()

<TemporaryDirectory '/tmp/tmpeb6bkv9m'>

In [51]:
import os ,shutil
def clear_folder(folder_path):
    if os.path.exists(folder_path):
        # Remove all the files in the directory
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            try:
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.unlink(file_path)
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
            except Exception as e:
                print(f"Failed to delete {file_path}. Reason: {e}")

In [52]:
os.sep

'/'

In [54]:
import streamlit as st

# Define the file uploader and submit button in a tab
def file_uploader_tab():
    st.header("File Uploader Tab")
    uploaded_file = st.file_uploader("Upload a file", type=["txt", "csv", "xlsx"])
    if uploaded_file is not None:
        st.write("File uploaded:", uploaded_file.name)
    if st.button("Submit"):
        st.write("Submit button clicked")

# Define the main page layout
def main_page():
    st.title("Main Page")
    
    # Row 1: Chat Interface
    st.subheader("Chat Interface")
    chat_placeholder = st.empty()
    user_message = st.text_input("Enter your message")
    if st.button("Send"):
        chat_placeholder.write(f"User: {user_message}")

    # Row 2: Placeholder for Text Message and Submit Button
    st.subheader("Text Message Input")
    text_message = st.text_area("Type your message here")
    if st.button("Submit Message"):
        st.write(f"Message Submitted: {text_message}")

    # Row 3: Click Button
    st.subheader("Click Button")
    if st.button("Click Me"):
        st.write("Button Clicked")

# Create the Streamlit app layout
def main():
    st.sidebar.title("Navigation")
    option = st.sidebar.selectbox("Choose a tab", ["Main Page", "File Uploader Tab"])
    
    if option == "Main Page":
        main_page()
    elif option == "File Uploader Tab":
        file_uploader_tab()

if __name__ == "__main__":
    main()


2024-09-10 23:06:19.209 
  command:

    streamlit run /home/srabinarayan/miniconda3/envs/genai_dev/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
2024-09-10 23:06:19.215 Session state does not function when running a script without `streamlit run`
